# Title
As a Manchester United supporter

For this analysis, I am using data from the Open Football project, which provides open public domain football data in CSV format (https://footballcsv.github.io/) for download. The Open Football project provides data from many leagues across the world, however, I will be focusing on the top division of English football for which data is provided between most of the years between 1888 to 2020.

This project focuses on answering the following:
- Does Manchester United live up to their reputation as comeback kings?
- How successful was the Sir Alex Ferguson era of Manchester United (1992-2012) compared to other historically great English eras like 70's Liverpool, 50's Everton


# Preparing the dataset

There are a number of wrinkles with the data that will require some adjustments before we can begin the analysis proper.

## Accessing the desired dataset(s)
Firstly, the download provides CSV files for each available season in separate folders - by decade, season, and then finally CSV files for each division. 

For example, the path to the CSV file containing results from the 2019-20 season of the English Premier League (the top division of English football) is:

> */path to folder/England-master/2010s/2019-20/eng.1.csv*

For flexibility, we want to be able to access any individual season for which there is data, as well as the ability to merge them into a single dataset.

For this purpose, we have a function that populates a dictionary with the correct paths for each CSV:

In [82]:
import pandas as pd
import numpy as np
import re

#This creates a dictionary with paths to each season's CSV file. You can modify the number of seasons by decade.
def seasons_dictionary(starting_season, ending_season):
    eng = 'D:\Google Drive\\New Dropbox\Work stuff\Professional Development\Data Projects\Football_python\england-master'
    seasons = {}
    starting_decade = starting_ #NOTE: START FIXING HERE
    ending_season = 
    for i in range(int(abs((starting_decade - ending_decade)/10)) + 1):
        decade = starting_decade
        decade_string = str(decade) + 's'
        list_of_seasons = []
        for i in range(10):
            season = str(decade + i) + '-' + str(decade + i + 1)[2:4]
            list_of_seasons.append(season)
            seasons[decade_string] = list_of_seasons
        starting_decade += 10
    df_dictionary = {}
    for decade in seasons:
        for season in seasons[decade]:
            path = eng + '\\' + decade + '\\' + season + '\eng.1.csv'
            df_name = 'df_eng_' + season
            df_dictionary[df_name] = path
    return df_dictionary

#To return the paths for every season in the 2010s:
seasons_dictionary(2010, 2010)

{'df_eng_2010-11': 'D:\\Google Drive\\New Dropbox\\Work stuff\\Professional Development\\Data Projects\\Football_python\\england-master\\2010s\\2010-11\\eng.1.csv',
 'df_eng_2011-12': 'D:\\Google Drive\\New Dropbox\\Work stuff\\Professional Development\\Data Projects\\Football_python\\england-master\\2010s\\2011-12\\eng.1.csv',
 'df_eng_2012-13': 'D:\\Google Drive\\New Dropbox\\Work stuff\\Professional Development\\Data Projects\\Football_python\\england-master\\2010s\\2012-13\\eng.1.csv',
 'df_eng_2013-14': 'D:\\Google Drive\\New Dropbox\\Work stuff\\Professional Development\\Data Projects\\Football_python\\england-master\\2010s\\2013-14\\eng.1.csv',
 'df_eng_2014-15': 'D:\\Google Drive\\New Dropbox\\Work stuff\\Professional Development\\Data Projects\\Football_python\\england-master\\2010s\\2014-15\\eng.1.csv',
 'df_eng_2015-16': 'D:\\Google Drive\\New Dropbox\\Work stuff\\Professional Development\\Data Projects\\Football_python\\england-master\\2010s\\2015-16\\eng.1.csv',
 'df_eng_2

Using the dictionary produced by this function, we can then specify any individual season (CSV file) to read in to a pandas dataframe.

In [83]:
#Creates a data frame from the CSV file of the specified season
def create_df(season):
    all_seasons = seasons_dictionary(1880, 2010)
    all_seasons['df_eng_' + season]
    df = pd.read_csv(all_seasons['df_eng_' + season])
    return df

#For example, here's a snapshot of the 2014-15 Premier League season:
EPL_14_15 = create_df('2014-15')
EPL_14_15.head(3)

,Round,Date,Team 1,FT,HT,Team 2
0,?,(Sat) 16 Aug 2014 (W33),Arsenal FC (1),2-1,1-1,Crystal Palace FC (1)
1,?,(Sat) 16 Aug 2014 (W33),Leicester City FC (1),2-2,1-2,Everton FC (1)
2,?,(Sat) 16 Aug 2014 (W33),Manchester United FC (1),1-2,0-1,Swansea City AFC (1)


We also want to be able to look at more than a single season, even though the data is contained in separate CSV files. Building on the seasons_dictionary() function again, we can read in and merge the CSV files for multiple seasons. Here, we use the merge_df() function below to create a data frame containing *all* the available data for top flight English football, from 1888 to present.

Note that not every year has data. There were interruptions to professional football, most notably during the years of the World Wars. The function has accommodated for this with error handling.

In [84]:
#Finally, this function reads in any number of season datasets as specified in the argument, and merges them into a single dataframe
def merge_df(starting_decade, ending_decade):
    df_dictionary = seasons_dictionary(starting_decade,ending_decade)
    df_list = []
    for i in df_dictionary:
        try:
            df = pd.read_csv(df_dictionary[i], index_col=None, header=0)
            df_list.append(df)
        except FileNotFoundError: #This exception accommodates the fact that data is not available for all years - for example, seasons that were not played due to WW1 and WW2.
            continue 
    merged_df = pd.concat(df_list, axis =0, ignore_index=True)
    return merged_df

#Let's make use of all the available data, from 1888 - present
merged_df = merge_df(1880, 2010)
merged_df

,Round,Date,Team 1,FT,HT,Team 2
0,?,(Sat) 8 Sep 1888 (36),Bolton Wanderers FC (1),3-6,?,Derby County FC (1)
1,?,(Sat) 8 Sep 1888 (36),Wolverhampton Wanderers FC (1),1-1,?,Aston Villa FC (1)
2,?,(Sat) 8 Sep 1888 (36),Preston North End FC (1),5-2,?,Burnley FC (1)
3,?,(Sat) 8 Sep 1888 (36),Everton FC (1),2-1,?,Accrington FC (1878-1896) (1)
4,?,(Sat) 8 Sep 1888 (36),Stoke City FC (1),0-2,?,West Bromwich Albion FC (1)
...,...,...,...,...,...,...
48986,?,(Sat) 7 Mar 2020 (W10),Wolverhampton Wanderers FC (29),0-0,0-0,Brighton & Hove Albion FC (29)
48987,?,(Sat) 7 Mar 2020 (W10),Burnley FC (29),1-1,1-0,Tottenham Hotspur FC (29)
48988,?,(Sun) 8 Mar 2020 (W10),Chelsea FC (29),4-0,2-0,Everton FC (29)
48989,?,(Sun) 8 Mar 2020 (W10),Manchester United FC (29),2-0,1-0,Manchester City FC (28)


As you can see, the merged dataset starts on the 8th September 1888 and it ends 48,991 games and 132 years later, with a game on 9th March 2020.

## Cleaning and enhancing the dataset

Now that we have the desired dataset, there is some cleaning to do:

- The dataset has missing data in the "Round" and "HT" columns. They are marked by the '?' symbol.
- The FT (full time) and HT (half time) scores are represented as strings, not numbers
- We must infer the goals scored by the home and away teams by the order of the string. '4-0' means the home team scored 4 goals to the away team scored 0 goals.

It would be nice (and much more useful) to be able to deal with these scores as numbers, and to have the goals explicitly attributed to the home or away teams.

To clean and enhance the dataset, we will need to:
- Get rid of the "Round" column - the vast majority of seasons have missing data for this column, and it is useless for our analysis.
- Add some columns which provide ways for us to cut the data, such as the number of goals scored by the home vs. away teams, the total number of goals scored, as well as goals scored in the first half vs. the second half.
- Find a way to handle the missing half-time scores when it comes to statistics which rely on that information.

In [86]:
#Let's make some changes to the dataset to better suit our purposes.
def add_columns(dataframe):
    df = dataframe
    #df['HT'].loc[df['HT'] == '?'] = np.NaN #Because there thousands of rows without HT score data (marked as '?'), this line converts these all to missing data ("NaN")
    #This converts a string '2-1' to a list [2,1] for the FT and HT columns
    FT_score_list = df['FT'].map(lambda FT: re.findall(r'\d+', FT)) 
    HT_score_list = df['HT'].map(lambda HT: re.findall(r'\d+', HT))
    #Now that we have the scores in a list format, we can convert them to integers and play with them to create the following additional statistics:
    df['Total goals'] = FT_score_list.map(lambda goals: int(goals[0]) + int(goals[1]))
    df['Home goals'] = FT_score_list.map(lambda goals: int(goals[0]))
    df['Away goals'] = FT_score_list.map(lambda goals: int(goals[1])) 
    df['First half goals'] = HT_score_list.map(lambda goals: int(goals[0]) + int(goals[1]) if goals != [] else np.NaN) #NOTE:the if condition is necessary because for many games the HT result is missing. They are marked as NaN if so.
    df['Second half goals'] = df['Total goals'] - df['First half goals']
    #Finally, delete the useless "round" column
    df = df.drop(columns=['Round'])
    return df

merged_df = add_columns(df)
merged_df

,Date,Team 1,FT,HT,Team 2,Total goals,Home goals,Away goals,First half goals,Second half goals
0,(Sat) 8 Sep 1888 (36),Bolton Wanderers FC (1),3-6,?,Derby County FC (1),9,3,6,NaN,NaN
1,(Sat) 8 Sep 1888 (36),Wolverhampton Wanderers FC (1),1-1,?,Aston Villa FC (1),2,1,1,NaN,NaN
2,(Sat) 8 Sep 1888 (36),Preston North End FC (1),5-2,?,Burnley FC (1),7,5,2,NaN,NaN
3,(Sat) 8 Sep 1888 (36),Everton FC (1),2-1,?,Accrington FC (1878-1896) (1),3,2,1,NaN,NaN
4,(Sat) 8 Sep 1888 (36),Stoke City FC (1),0-2,?,West Bromwich Albion FC (1),2,0,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
48986,(Sat) 7 Mar 2020 (W10),Wolverhampton Wanderers FC (29),0-0,0-0,Brighton & Hove Albion FC (29),0,0,0,0.0,0.0
48987,(Sat) 7 Mar 2020 (W10),Burnley FC (29),1-1,1-0,Tottenham Hotspur FC (29),2,1,1,1.0,1.0
48988,(Sun) 8 Mar 2020 (W10),Chelsea FC (29),4-0,2-0,Everton FC (29),4,4,0,2.0,2.0
48989,(Sun) 8 Mar 2020 (W10),Manchester United FC (29),2-0,1-0,Manchester City FC (28),2,2,0,1.0,1.0


# Analysing the dataset

With a large historical dataset and some additional columns of information, we can now begin exploring.

## Filtering the data

Again, it would be useful to have some way to quickly isolate certain information to make this exploration easier. It would be great if we could:
- Look up a particular team's fixtures
- Look up particular matchups between two teams
- Get some descriptive statistics about those filtered results, such as average number of goals scored


In [87]:
#Look up a particular team's fixtures
def find_team_games(df, name):
    games = df.loc[(df['Team 1'].str.contains(name, re.compile('(\d)|(\d\d)')) == True) | (df['Team 2'].str.contains(name, re.compile('(\d)|(\d\d)')) == True)]
    return games

#Let's find Manchester United results
find_team_games(merged_df,'Manchester United')

,Date,Team 1,FT,HT,Team 2,Total goals,Home goals,Away goals,First half goals,Second half goals
584,(Sat) 3 Sep 1892 (35),Blackburn Rovers FC (1),4-3,?,Manchester United FC (1),7,4,3,NaN,NaN
592,(Sat) 10 Sep 1892 (36),Manchester United FC (2),1-1,?,Burnley FC (3),2,1,1,NaN,NaN
600,(Sat) 17 Sep 1892 (37),Burnley FC (4),4-1,?,Manchester United FC (3),5,4,1,NaN,NaN
606,(Sat) 24 Sep 1892 (38),Everton FC (4),6-0,?,Manchester United FC (4),6,6,0,NaN,NaN
619,(Sat) 1 Oct 1892 (39),West Bromwich Albion FC (5),0-0,?,Manchester United FC (5),0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
48950,(Sat) 1 Feb 2020 (W5),Manchester United FC (25),0-0,0-0,Wolverhampton Wanderers FC (25),0,0,0,0.0,0.0
48961,(Mon) 17 Feb 2020 (W8),Chelsea FC (26),0-2,0-1,Manchester United FC (26),2,0,2,1.0,1.0
48969,(Sun) 23 Feb 2020 (W8),Manchester United FC (27),3-0,1-0,Watford FC (27),3,3,0,1.0,2.0
48979,(Sun) 1 Mar 2020 (W9),Everton FC (28),1-1,1-1,Manchester United FC (28),2,1,1,2.0,0.0


In [88]:
def find_matchups(df, team1, team2):
    matchups = df.loc[(df['Team 1'].str.contains((team1), re.compile('(\d)|(\d\d)')) == True) & (df['Team 2'].str.contains((team2), re.compile('(\d)|(\d\d)')) == True) | (df['Team 1'].str.contains((team2), re.compile('(\d)|(\d\d)')) == True) & (df['Team 2'].str.contains((team1), re.compile('(\d)|(\d\d)')) == True)]
    return matchups

#Let's look at the Merseyside derby
find_matchups(merged_df, 'Manchester United', 'Liverpool')

,Date,Team 1,FT,HT,Team 2,Total goals,Home goals,Away goals,First half goals,Second half goals
4725,(Tue) 25 Dec 1906 (52),Manchester United FC (19),0-0,?,Liverpool FC (19),0,0,0,NaN,NaN
4867,(Mon) 1 Apr 1907 (13),Liverpool FC (33),0-1,?,Manchester United FC (34),1,0,1,NaN,NaN
4934,(Sat) 7 Sep 1907 (35),Manchester United FC (2),4-0,?,Liverpool FC (2),4,4,0,NaN,NaN
5228,(Wed) 25 Mar 1908 (12),Liverpool FC (30),7-4,?,Manchester United FC (29),11,7,4,NaN,NaN
5345,(Sat) 26 Sep 1908 (38),Manchester United FC (5),3-2,?,Liverpool FC (6),5,3,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
48236,(Sat) 10 Mar 2018 (W10),Manchester United FC (30),2-1,2-0,Liverpool FC (30),3,2,1,2.0,1.0
48491,(Sun) 16 Dec 2018 (W50),Liverpool FC (17),3-1,1-1,Manchester United FC (17),4,3,1,2.0,2.0
48591,(Sun) 24 Feb 2019 (W8),Manchester United FC (27),0-0,0-0,Liverpool FC (27),0,0,0,0.0,0.0
48791,(Sun) 20 Oct 2019 (W42),Manchester United FC (9),1-1,1-0,Liverpool FC (9),2,1,1,1.0,1.0


In [89]:
def describe_team(dataframe, team):
    df = dataframe
    games = df['Home goals'].loc[(df['Team 1'].str.contains(team, re.compile('(\d)|(\d\d)')) == True) | (df['Team 2'].str.contains(team, re.compile('(\d)|(\d\d)')) == True)].describe()
    #away = df['Home goals'].loc[(df['Team 1'].str.contains(team, re.compile('(\d)|(\d\d)')) == True) | (df['Team 2'].str.contains(team, re.compile('(\d)|(\d\d)')) == True)].sum()
    return games

describe_team(merged_df, 'Manchester United')

count    3810.000000
mean        1.778215
std         1.447750
min         0.000000
25%         1.000000
50%         2.000000
75%         3.000000
max        10.000000
Name: Home goals, dtype: float64